In [1]:
import pandas as pd
from gazpacho import get, Soup # Import gazpacho

url = "https://tax.alaska.gov/programs/oil/production/ans.aspx?" # Set the primary url value as a "string"

html = get(url) # Get the html from the url
soup = Soup(html) # Pass html into soup
list = soup.find('option')[1:] # Find all the 'option' html tags and add to list
length = len(list) # Number of records in the list (month/year)

urls = [] # Create an empty list for all the urls

# 1. Scrape the dropdown selection menu for website urls containing html data tables

for i in range(length): # Loop through all the html tags for 'options' for the length of the list
    options = list[i].attrs['value'] # Create variable and set it to the 'value' in the options list
#     print(url + options) # Print a list of urls to be scraped
    urls.append(url + options) # and add the value to the urls list
    
url_count = len(urls) # Number of urls in the list

# 2. Scrape the website urls containing html data tables

for url in urls:
    
    file_df = pd.read_csv('Resources/alaska_oil_production.csv') # Read previously scraped data file into Pandas
    file_df['Date'] = pd.to_datetime(file_df['Date'], errors='coerce') # Convert column to datetime datatype
    
    scraped_df = pd.read_html(url)[6] # Read the 7th data table from each url
    scraped_df = pd.DataFrame(scraped_df) # Set scraped_df as a dataframe

    scraped_df.columns = file_df.columns # Set the columns name from the file equal to the scraped_df

    scraped_df['Date'] = pd.to_datetime(scraped_df['Date'], errors='coerce') # Convert the 'Date' column to datetime datatype
    scraped_df = scraped_df.dropna(axis=0, how='any', subset=['Date']) # Drop any rows with NaN in 'Date'

    merged_df = file_df.append(scraped_df) # Merge the scraped_df to the file_df
    merged_df = merged_df.sort_values(by='Date', ascending=False) # Sort by 'Date' in descending order
    merged_df['duplicates'] = merged_df.duplicated('Date', keep='last') # Create a column to assign duplicates T/F
    merged_df = merged_df[merged_df['duplicates'] == False] # Save df with duplicates equal to False only
    merged_df = merged_df.drop(columns=['duplicates'])# Delete the 'duplicates' column if dupe_count = 0
    merged_df = merged_df.reset_index(drop=True) # Reset the merged dataframe index
    merged_df.to_csv('Resources/alaska_oil_production.csv', mode='w', index=False) # Write merged dataframe to existing csv

In [2]:
merged_df.head()

,Date,Prudhoe Bay,Kuparuk,Endicott,Lisburne,Alpine,ANS,Inventories,Milne Point,Northstar,Cook Inlet,Alaska,PS# Temperature
0,2021-11-30,307793.0,106087.0,0.0,21192.0,49749.0,484821.0,3222900.0,NaN,NaN,NaN,NaN,NaN
1,2021-11-29,335457.0,111403.0,0.0,21241.0,32965.0,501066.0,4062313.0,NaN,NaN,NaN,NaN,NaN
2,2021-11-28,330968.0,108272.0,0.0,21177.0,48201.0,508618.0,4264480.0,NaN,NaN,NaN,NaN,NaN
3,2021-11-27,332692.0,104800.0,0.0,21507.0,47830.0,506829.0,3773589.0,NaN,NaN,NaN,NaN,NaN
4,2021-11-26,335262.0,103336.0,0.0,21525.0,48318.0,508442.0,3280298.0,NaN,NaN,NaN,NaN,NaN
